In [ ]:
from dlgo.data.processor import GoDataProcessor
from dlgo.encoders.sevenplane import SevenPlaneEncoder

from keras.models import Sequential
from keras.callbacks import ModelCheckpoint  # <1>
# <1> With model checkpoints we can store progress for time-consuming experiments
 
if __name__ == '__main__':
    go_board_rows, go_board_cols = 19, 19
    num_classes = go_board_rows * go_board_cols
    num_games = 100

    encoder = SevenPlaneEncoder((go_board_rows, go_board_cols))  # <1>

    processor = GoDataProcessor(encoder=encoder.name())  # <2>

    X, Y = processor.load_go_data('train', num_games)  # <3>
    print('X.shape', X.shape)
    print('Y.shape', Y.shape)

    validationX, validationY = processor.load_go_data('test', num_games)
    print('validationX.shape', validationX.shape)
    print('validationY.shape', validationY.shape)
    # <1> First we create an encoder of board size.
    # <2> Then we initialize a Go Data processor with it.
    # <3> From the processor we create two data generators, for training and testing.


ImportError: cannot import name 'SevenPlaneEncoder' from 'dlgo.encoders.oneplane' (/Users/regis.cridlig/Development/Python/deep_learning_and_the_game_of_go/code/dlgo/encoders/oneplane.py)

In [ ]:
    from dlgo.networks import small
    from keras.layers import Dense
    from keras.optimizers import Adagrad

    input_shape = (encoder.num_planes, go_board_rows, go_board_cols)
    network_layers = small.layers(input_shape)
    model = Sequential()
    for layer in network_layers:
        model.add(layer)
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer=Adagrad(), metrics=['accuracy'])
    print(model.summary())

    epochs = 5
    batch_size = 256
    model.fit(X, Y, batch_size=batch_size,
                    epochs=epochs,
                    validation_data=(validationX, validationY),
                    verbose=1,
                    callbacks=[ModelCheckpoint('../checkpoints/small_model_epoch_{epoch}.keras')])  # <5>
    # <5> After each epoch we persist a checkpoint of the model.
    model.evaluate(validationX, validationY, batch_size=batch_size)  # <6>
    # <6> For evaluation we also specify the validation set.

2024-11-28 23:45:14.941751: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Pro
2024-11-28 23:45:14.941936: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-11-28 23:45:14.941952: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-11-28 23:45:14.942169: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-11-28 23:45:14.942506: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ zero_padding2d (ZeroPadding2D)  │ (None, 1, 25, 25)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 48, 19, 19)     │         2,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu (LeakyReLU)         │ (None, 48, 19, 19)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ zero_padding2d_1                │ (None, 48, 23, 23)     │             0 │
│ (ZeroPadding2D)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 32, 19, 19)     │        38,432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_1 (LeakyReLU)       │ (None, 32, 19, 19)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ zero_padding2d_2                │ (None, 32, 23, 23)     │             0 │
│ (ZeroPadding2D)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 32, 19, 19)     │        25,632 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_2 (LeakyReLU)       │ (None, 32, 19, 19)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ zero_padding2d_3                │ (None, 32, 23, 23)     │             0 │
│ (ZeroPadding2D)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 32, 19, 19)     │        25,632 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_3 (LeakyReLU)       │ (None, 32, 19, 19)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 11552)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │     5,915,136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_4 (LeakyReLU)       │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 361)            │       185,193 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,192,425 (23.62 MB)

 Trainable params: 6,192,425 (23.62 MB)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/5


2024-11-28 23:45:24.860715: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


15328/15328 ━━━━━━━━━━━━━━━━━━━━ 788s 51ms/step - accuracy: 0.0039 - loss: 5.8468 - val_accuracy: 0.0094 - val_loss: 5.5931
Epoch 2/5
15328/15328 ━━━━━━━━━━━━━━━━━━━━ 793s 52ms/step - accuracy: 0.0098 - loss: 5.5478 - val_accuracy: 0.0118 - val_loss: 5.5098
Epoch 3/5
 9797/15328 ━━━━━━━━━━━━━━━━━━━━ 4:52 53ms/step - accuracy: 0.0130 - loss: 5.4806